In [ ]:
# Cell 1 (REVISED): Uninstall and Reinstall Key Libraries

# Step 1: Clean up any potentially conflicting versions
print("--> Uninstalling existing versions...")
!pip uninstall -y transformers accelerate datasets

# Step 2: Reinstall a stable, compatible set of the core libraries
print("\n--> Reinstalling core libraries...")
!pip install transformers accelerate datasets

# Step 3: Install the remaining libraries
print("\n--> Installing other required libraries...")
!pip install scikit-learn pandas imbalanced-learn -q

print("\n✅ All libraries have been reinstalled.")

--> Uninstalling existing versions...
Found existing installation: transformers 4.55.1
Uninstalling transformers-4.55.1:
  Successfully uninstalled transformers-4.55.1
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0

--> Reinstalling core libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:

In [ ]:
# <-- NEW: Check library versions
# This output should be noted for the 'reproducibility' section of the paper
!pip freeze | grep -E "transformers|torch|scikit-learn|imbalanced-learn|datasets"

datasets==4.0.0
imbalanced-learn==0.13.0
scikit-learn==1.6.1
sentence-transformers==5.1.0
tensorflow-datasets==4.9.9
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
transformers==4.55.2
vega-datasets==0.9.0


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import os
import json
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# --- KEY SETTINGS ---
SEED = 42
GDRIVE_PATH = '/content/drive/MyDrive/eecsi_revise/'
MODEL_NAME = "indolem/indobertweet-base-uncased"

# Set seed for reproducibility across all libraries
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print(f"✅ Setup complete. Working inside folder: {GDRIVE_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Setup complete. Working inside folder: /content/drive/MyDrive/eecsi_revise/


In [ ]:
# Define file paths
file_path_csv = os.path.join(GDRIVE_PATH, 'final_golden_dataset_eecsi.csv')
file_path_split = os.path.join(GDRIVE_PATH, 'kfold_splits.pkl')

# Load the dataset and the split file
try:
    df = pd.read_csv(file_path_csv)
    with open(file_path_split, 'rb') as f:
        kfold_splits = pickle.load(f)
    print("✅ Successfully loaded dataset and 5-fold splits.")
except FileNotFoundError as e:
    print(f"❌ ERROR: File not found. Please ensure '{e.filename}' is in the 'eecsi_revise' folder in your Google Drive.")

✅ Successfully loaded dataset and 5-fold splits.


In [ ]:
# Cell 5: Prepare Helper Functions & Classes

# 1. Custom PyTorch Dataset Class
class AspectDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 2. Function to compute metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    report = classification_report(p.label_ids, preds, output_dict=True, zero_division=0)
    # The labels parameter is needed for a consistent report structure
    class_labels = [id2label[i] for i in range(len(id2label))]
    detailed_report = classification_report(p.label_ids, preds, labels=list(range(len(class_labels))), target_names=class_labels, output_dict=True, zero_division=0)
    return {"macro_f1": detailed_report["macro avg"]["f1-score"], "detailed_report": detailed_report}

# 3. Custom Trainer for Weighted Loss
class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # --- THE CHANGE IS HERE: Added **kwargs ---
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Move class_weights to the same device as logits
        weights = self.class_weights.to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

print("✅ Helper functions and classes are ready.")

✅ Helper functions and classes are ready.


In [ ]:
# List to store the evaluation results from each fold
fold_results = []
# Create label mappings for consistency
labels = np.array(sorted(df['aspect'].unique()))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

for i, fold in enumerate(kfold_splits):
    print(f"--- Running Fold {i+1}/5 ---")

    # 1. Split data for the current fold
    train_index, test_index = fold['train'], fold['test']
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    # 2. Apply Random Oversampling ONLY on the training data
    ros = RandomOverSampler(random_state=SEED)
    X_train_resampled, y_train_resampled = ros.fit_resample(train_df[['cleaned_text']], train_df['aspect'])
    train_df_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)

    # 3. Compute Class Weights from the ORIGINAL imbalanced training data for this fold
    class_weights = compute_class_weight('balanced', classes=labels, y=train_df['aspect'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

    # 4. Tokenize data
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_encodings = tokenizer(list(train_df_resampled['cleaned_text']), truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(list(test_df['cleaned_text']), truncation=True, padding=True, max_length=128)

    # Convert labels to integers
    train_labels = [label2id[label] for label in train_df_resampled['aspect']]
    test_labels = [label2id[label] for label in test_df['aspect']]

    # Create PyTorch Datasets
    train_dataset = AspectDataset(train_encodings, train_labels)
    test_dataset = AspectDataset(test_encodings, test_labels)

    # 5. Initialize the model for each fold to prevent weight leakage
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(labels),
        label2id=label2id,
        id2label=id2label
    )

    # <-- NEW: Print model parameter count (only for the first fold)
    if i == 0:
        print(f"Model Parameters: {model.num_parameters()/1e6:.2f}M")

    # 6. Define Training Arguments
    training_args = TrainingArguments(
        output_dir=os.path.join(GDRIVE_PATH, f'results_fold_{i+1}'),
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        seed=SEED,
    )

    # 7. Use the custom WeightedLossTrainer
    trainer = WeightedLossTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor,
    )

    # 8. Train the model
    trainer.train()

    # 9. Evaluate and store the results
    eval_results = trainer.evaluate()
    fold_results.append(eval_results)
    print(f"Fold {i+1} complete. Evaluation results: {eval_results}")

print("\n✅ 5-fold cross-validation process for IndoBERT finished.")

--- Running Fold 1/5 ---


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro F1,Detailed Report
1,0.308400,0.918843,0.702426,"{np.str_('Irrelevant'): {'precision': 0.8609271523178808, 'recall': 0.6565656565656566, 'f1-score': 0.7449856733524355, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6, 'recall': 0.8360655737704918, 'f1-score': 0.6986301369863014, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7619047619047619, 'recall': 0.9056603773584906, 'f1-score': 0.8275862068965517, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7678571428571429, 'recall': 0.5972222222222222, 'f1-score': 0.671875, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4788732394366197, 'recall': 0.5862068965517241, 'f1-score': 0.5271317829457365, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.75, 'recall': 0.9056603773584906, 'f1-score': 0.8205128205128205, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.5166666666666667, 'recall': 0.7948717948717948, 'f1-score': 0.6262626262626263, 'support': 39.0}, 'accuracy': 0.7062706270627063, 'macro avg': {'precision': 0.6766041375975816, 'recall': 0.7546075569569816, 'f1-score': 0.7024263209937817, 'support': 606.0}, 'weighted avg': {'precision': 0.7354630445522199, 'recall': 0.7062706270627063, 'f1-score': 0.7082850428510536, 'support': 606.0}}"
2,0.133300,0.924008,0.729954,"{np.str_('Irrelevant'): {'precision': 0.8125, 'recall': 0.7878787878787878, 'f1-score': 0.8, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6617647058823529, 'recall': 0.7377049180327869, 'f1-score': 0.6976744186046512, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7966101694915254, 'recall': 0.8867924528301887, 'f1-score': 0.8392857142857143, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7412587412587412, 'recall': 0.7361111111111112, 'f1-score': 0.7386759581881533, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.5686274509803921, 'recall': 0.5, 'f1-score': 0.5321100917431193, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8214285714285714, 'recall': 0.8679245283018868, 'f1-score': 0.8440366972477065, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6756756756756757, 'recall': 0.6410256410256411, 'f1-score': 0.6578947368421053, 'support': 39.0}, 'accuracy': 0.7491749174917491, 'macro avg': {'precision': 0.7254093306738941, 'recall': 0.7367767770257718, 'f1-score': 0.7299539452730643, 'support': 606.0}, 'weighted avg': {'precision': 0.7476430735595075, 'recall': 0.7491749174917491, 'f1-score': 0.7476301078930563, 'support': 606.0}}"
3,0.035200,1.081516,0.745827,"{np.str_('Irrelevant'): {'precision': 0.8546511627906976, 'recall': 0.7424242424242424, 'f1-score': 0.7945945945945946, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.7368421052631579, 'recall': 0.6885245901639344, 'f1-score': 0.711864406779661, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.8070175438596491, 'recall': 0.8679245283018868, 'f1-score': 0.8363636363636363, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.6946107784431138, 'recall': 0.8055555555555556, 'f1-score': 0.7459807073954984, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6470588235294118, 'recall': 0.5689655172413793, 'f1-score': 0.6055045871559633, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8518518518518519, 'recall': 0.8679245283018868, 'f1-score': 0.8598130841121495, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6041666666666666, 'recall': 0.7435897435897436, 'f1-score': 0.6666666666666666, 'support': 39.0}, 'accuracy': 0.7574257425742574, 'macro avg': {'precision': 0.7423141332006499, 'recall': 0.7549869579398042, 'f1-score': 0.7458268118668814, 'support': 606.0}, 'weighted avg': {'precision': 0.7643634331465917, 'recall': 0.7574257425742574, 'f1-score': 0.757741440014032, 'support': 606.0}}"
4,0.007200,1.144598,0.750295,"{np.str_('Irrelevant'): {'precision': 0.851063829787234, 'recall': 0

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.8609271523178808, 'recall': 0.6565656565656566, 'f1-score': 0.7449856733524355, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6, 'recall': 0.8360655737704918, 'f1-score': 0.6986301369863014, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7619047619047619, 'recall': 0.9056603773584906, 'f1-score': 0.8275862068965517, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7678571428571429, 'recall': 0.5972222222222222, 'f1-score': 0.671875, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4788732394366197, 'recall': 0.5862068965517241, 'f1-score': 0.5271317829457365, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.75, 'recall': 0.9056603773584906, 'f1-score': 0.8205128205128205, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.5166666666666667, 'recall': 0.7948717948717948, 'f1-score': 0.6262626262626263, 'support': 39.0}, 'accuracy': 

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.851063829787234, 'recall': 0.8080808080808081, 'f1-score': 0.8290155440414507, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6935483870967742, 'recall': 0.7049180327868853, 'f1-score': 0.6991869918699187, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7833333333333333, 'recall': 0.8867924528301887, 'f1-score': 0.831858407079646, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7586206896551724, 'recall': 0.7638888888888888, 'f1-score': 0.7612456747404844, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6101694915254238, 'recall': 0.6206896551724138, 'f1-score': 0.6153846153846154, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8653846153846154, 'recall': 0.8490566037735849, 'f1-score': 0.8571428571428571, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.65, 'recall': 0.6666666666666666, 'f1-score': 0.6582278481012658, 'suppor

Fold 5 complete. Evaluation results: {'eval_loss': 1.1445977687835693, 'eval_macro_f1': 0.7502945626228911, 'eval_detailed_report': {np.str_('Irrelevant'): {'precision': 0.851063829787234, 'recall': 0.8080808080808081, 'f1-score': 0.8290155440414507, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6935483870967742, 'recall': 0.7049180327868853, 'f1-score': 0.6991869918699187, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7833333333333333, 'recall': 0.8867924528301887, 'f1-score': 0.831858407079646, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7586206896551724, 'recall': 0.7638888888888888, 'f1-score': 0.7612456747404844, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6101694915254238, 'recall': 0.6206896551724138, 'f1-score': 0.6153846153846154, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8653846153846154, 'recall': 0.8490566037735849, 'f1-score': 0.8571428571428571, 'support': 53.0}, np.str_('Smart People')

In [ ]:
# Extract the 'eval_macro_f1' score from each fold's result
macro_f1_scores = [result['eval_macro_f1'] for result in fold_results]

# Calculate the mean and standard deviation
mean_macro_f1 = np.mean(macro_f1_scores)
std_macro_f1 = np.std(macro_f1_scores)

print("--- Final Aggregated Results (5-Fold CV) for IndoBERTweet (Improved) ---")
print(f"Macro F1-Score = {mean_macro_f1:.4f} ± {std_macro_f1:.4f}")

--- Final Aggregated Results (5-Fold CV) for IndoBERTweet (Improved) ---
Macro F1-Score = 0.7569 ± 0.0102


In [ ]:
# Prepare the results dictionary for saving
final_results = {
    'model': 'IndoBERTweet',
    'mean_macro_f1': mean_macro_f1,
    'std_dev_macro_f1': std_macro_f1,
    'results_per_fold': fold_results
}

# Define the output file path
results_file_path = os.path.join(GDRIVE_PATH, 'results_indobertweet_asc.json')

# Save to a JSON file
with open(results_file_path, 'w') as f:
    json.dump(final_results, f, indent=4)

print(f"\n✅ Final results for IndoBERTweet have been saved to: '{results_file_path}'")


✅ Final results for IndoBERTweet have been saved to: '/content/drive/MyDrive/eecsi_revise/results_indobertweet.json'
